# QB Resilience Metric Engineering

The goal, in short, is to figure out factors like OL play (and injuries), QB # of historical snaps entering game, etc.

-rather than drive-based, game- and prior-game based
    
        -(or can agg previous drives' pass rate, really, using modified script)
                -can also do pressure over season, qb-induce-pressure tendencey
        -play-action tendency, box count faced tendency in past for qb?
                -best way to bolster PA indicator is past PA indications

-can see if we incorp. IPA stats and other stuff not quite working elsewhere

tempo: run success rate vs pass success rate?

combine regular pass rate, xpass?

guiding q: how does it help pred. pass?

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import chain
import nfl_data_py as nfl
from utility_db_25 import get_momentum_cols, create_momentum_index, get_final_features, get_motion_cols, motion_complexity_score

# Load data

We load train & play data, then flag man/zone discrepancies for df_play

In [2]:
root_dir = os.getcwd()
train_data=pd.read_csv(os.path.join(root_dir, "data/train_data.csv")).sort_values(by=['gameId','possessionTeam','playId'])
df_play = pd.read_csv(os.path.join(root_dir,'data/plays.csv')).sort_values(by=['gameId','possessionTeam','playId'])
df_player_play = pd.read_csv(os.path.join(root_dir,'data/player_play.csv'))
df_players = pd.read_csv(os.path.join(root_dir,'data/players.csv'))

Label man/zone

In [3]:
df_play = pd.read_csv(os.path.join(root_dir,'data/plays.csv')).sort_values(by=['gameId','possessionTeam','playId'])
df_play['pff_manZone'] = df_play['pff_manZone'].fillna('UNK')
df_play['is_man'] = 0
df_play.loc[df_play['pff_manZone'] == 'Man','is_man'] = 1

### Get rolling man/zone ratios

We first start by getting rolling man-defense counts; we also number each play of the game (based on possession team)

In [4]:
df_play = pd.concat([df_play,df_play.groupby(['gameId','possessionTeam']).agg(man_ct_game=('is_man','cumsum'),off_play=('is_man','cumcount'))],axis=1)
df_play['off_play']+=1

We now use this data to, for each play, get the ratio of man coverage used to that point:

In [5]:
df_play['man_ratio_pre'] = df_play['man_ct_game']/df_play['off_play']
df_play['man_ratio'] = df_play.groupby(['gameId','possessionTeam']).man_ratio_pre.shift(1)
df_play['man_ratio'] = df_play['man_ratio'].fillna(.2)
df_play[['gameId','playId','possessionTeam','is_man','man_ct_game','off_play','man_ratio_pre','man_ratio']].head(6)

,gameId,playId,possessionTeam,is_man,man_ct_game,off_play,man_ratio_pre,man_ratio
13368,2022090800,56,BUF,0,0,1,0.0,0.2
2660,2022090800,80,BUF,0,0,2,0.0,0.0
7042,2022090800,101,BUF,0,0,3,0.0,0.0
2909,2022090800,122,BUF,0,0,4,0.0,0.0
10143,2022090800,167,BUF,0,0,5,0.0,0.0
5958,2022090800,191,BUF,0,0,6,0.0,0.0


Incorporate man ratio into train data:

In [6]:
train_data = train_data.merge(df_play[['gameId','playId','man_ratio']],how='left')

### Get box EWM

We get the exponential weighted mean for box-count now:

In [7]:
train_data['box_ewm_pre'] = train_data.groupby(['gameId','possessionTeam'])['n_defense_box'].transform(lambda x: x.ewm(alpha=.1).mean())
train_data['box_ewm'] = train_data.groupby(['gameId','possessionTeam']).box_ewm_pre.shift(1)

In [9]:
train_data[['gameId','playId','possessionTeam','n_defense_box','box_ewm']].head(6)

,gameId,playId,possessionTeam,n_defense_box,box_ewm
0,2022090800,56,BUF,6.0,NaN
1,2022090800,80,BUF,6.0,6.000000
2,2022090800,101,BUF,7.0,6.000000
3,2022090800,122,BUF,6.0,6.369004
4,2022090800,167,BUF,5.0,6.261704
5,2022090800,191,BUF,6.0,5.953603


Note: is excised now, but ewm significantly outperforms (almost 2:1) just mean box count

In [12]:
train_data[['man_ratio','box_ewm','pass']].corr()

,man_ratio,box_ewm,pass
man_ratio,1.000000,0.134879,-0.009945
box_ewm,0.134879,1.000000,-0.088376
pass,-0.009945,-0.088376,1.000000


In [11]:
box_ewm_corrs = train_data.corr()['box_ewm'].sort_values(ascending=False)
weight_corrs = train_data.corr()['weight_all_sum'].sort_values(ascending=False)

In [13]:
box_ewm_corrs.head(25)

box_ewm                   1.000000
box_ewm_pre               0.935949
n_defense_box             0.254748
time_remaining            0.160234
man_ratio                 0.134879
weight_all_sum            0.130069
s|max|T_2                 0.112179
o_standard|std|T_2        0.108824
o_standard|mean|QB_1      0.108347
o_standard|mean|T_2       0.104709
y_standard|mean|T_2       0.102222
dis|sum|T_2               0.100384
humidity                  0.099808
dir_standard|std|T_2      0.098326
y_standard|std|T_2        0.093006
dir_standard|mean|TE_2    0.089463
x_standard|mean|TE_2      0.088521
dir_standard|std|TE_2     0.087405
o_standard|mean|TE_2      0.085919
Cover-2_def               0.085909
yac_att                   0.085630
y_standard|mean|TE_2      0.084659
a|max|T_2                 0.083191
o_standard|std|T_1        0.081955
s|max|TE_2                0.080551
Name: box_ewm, dtype: float64

In [14]:
weight_corrs.head(25)

weight_all_sum                 1.000000
o_standard|mean|TE_2           0.404034
y_standard|mean|TE_2           0.399727
x_standard|mean|TE_2           0.391052
dir_standard|std|TE_2          0.389062
dir_standard|mean|TE_2         0.387612
s|max|TE_2                     0.351831
n_defense_box                  0.351236
a|max|TE_2                     0.347494
x_standard|std|TE_2            0.345018
o_standard|std|TE_2            0.319780
dis|sum|TE_2                   0.312823
y_standard|std|TE_2            0.250187
n_offense_backfield            0.244578
x_standard|std|QB_1            0.241402
x_standard|mean|TE_3           0.235657
x_standard|mean|FB_1           0.233519
o_standard|mean|TE_3           0.233065
y_standard|mean|FB_1           0.231933
y_standard|mean|TE_3           0.229422
offenseFormation_SINGLEBACK    0.227644
dir_standard|std|FB_1          0.227638
dir_standard|mean|TE_3         0.227236
o_standard|mean|FB_1           0.224957
box_ewm_pre                    0.224176


# Process BMI data

First, we convert height to inches, then get BMI:

In [15]:
# calc height, bmi
df_players = pd.concat([df_players,df_players['height'].str.split('-',n=1,expand=True).rename(columns={0:'h_ft',1:'h_in_pre'})],axis=1)
df_players['height_inches'] = df_players['h_ft'].astype(int)*12 + df_players['h_in_pre'].astype(int)
df_players['bmi'] = df_players['weight'] /(df_players['height_inches']**2)

# incorporate data back into player-play
df_bmi = df_player_play[['gameId','playId','nflId']].merge(df_players[['nflId','bmi','height_inches','weight','position']])
df_bmi.head(1)

,gameId,playId,nflId,bmi,height_inches,weight,position
0,2022090800,56,35472,0.054815,77,325,G


### Get BMI by position

For each play, we get the mean BMI, weight, and height:

In [16]:
ol_df = df_bmi[df_bmi['position'].isin(['C','G','T'])].groupby(['gameId','playId'])[['weight','height_inches','bmi']].mean().reset_index().rename(columns={'weight':'mean_OL_weight','height_inches':'mean_OL_height','bmi':'mean_OL_bmi'})
dl_df = df_bmi[df_bmi['position'].isin(['DT','NT','DE'])].groupby(['gameId','playId'])[['weight','height_inches','bmi']].mean().reset_index().rename(columns={'weight':'mean_DL_weight','height_inches':'mean_DL_height','bmi':'mean_DL_bmi'})
lb_df = df_bmi[df_bmi['position'].isin(['LB','OLB','ILB'])].groupby(['gameId','playId'])[['weight','height_inches','bmi']].mean().reset_index().rename(columns={'weight':'mean_LB_weight','height_inches':'mean_LB_height','bmi':'mean_LB_bmi'})
cb_df = df_bmi[df_bmi['position'].isin(['CB'])].groupby(['gameId','playId'])[['weight','height_inches','bmi']].mean().reset_index().rename(columns={'weight':'mean_CB_weight','height_inches':'mean_CB_height','bmi':'mean_CB_bmi'})
wr_df = df_bmi[df_bmi['position'].isin(['WR'])].groupby(['gameId','playId'])[['weight','height_inches','bmi']].mean().reset_index().rename(columns={'weight':'mean_WR_weight','height_inches':'mean_WR_height','bmi':'mean_WR_bmi'})
te_df = df_bmi[df_bmi['position'].isin(['TE'])].groupby(['gameId','playId'])[['weight','height_inches','bmi']].mean().reset_index().rename(columns={'weight':'mean_TE_weight','height_inches':'mean_TE_height','bmi':'mean_TE_bmi'})

Then, we integrate all these positional BMI's:

In [17]:
df_play = df_play.merge(ol_df,how='left')
df_play = df_play.merge(dl_df,how='left')
df_play = df_play.merge(lb_df,how='left')
df_play = df_play.merge(cb_df,how='left')
df_play = df_play.merge(wr_df,how='left')
df_play = df_play.merge(te_df,how='left')

We next calculate rough "delta" BMI's between oppositional positions (e.g., WR/CB), as well as nominal 'box' data

In [18]:
df_play['wr_cb_bmi_delta'] = df_play['mean_WR_bmi']-df_play['mean_CB_bmi']
df_play['ol_dl_bmi_delta'] = df_play['mean_OL_bmi']-df_play['mean_DL_bmi']
df_play['ol_box_delta'] = df_play['mean_OL_bmi']-((df_play['mean_DL_bmi'] + df_play['mean_LB_bmi']) /2)
df_play['ol_plus_box_delta'] = ((df_play['mean_OL_bmi']+df_play['mean_TE_bmi'])/2)-((df_play['mean_DL_bmi'] + df_play['mean_LB_bmi']) /2)
df_play['box_weight'] = (df_play['mean_DL_weight'] + df_play['mean_LB_weight']) /2
df_play['box_bmi'] = (df_play['mean_DL_bmi'] + df_play['mean_LB_bmi']) /2

In [19]:
train_data = train_data.merge(df_play[['gameId','playId']+list(df_play.columns[-28:])],how='left')

In [20]:
train_data[list(df_play.columns[-18:]) + ['pass']].corr()['pass']

mean_LB_weight       0.041468
mean_LB_height       0.030665
mean_LB_bmi          0.014612
mean_CB_weight      -0.036760
mean_CB_height      -0.055954
mean_CB_bmi          0.016943
mean_WR_weight      -0.033220
mean_WR_height      -0.044301
mean_WR_bmi          0.005789
mean_TE_weight      -0.078581
mean_TE_height      -0.035519
mean_TE_bmi         -0.048478
wr_cb_bmi_delta     -0.003739
ol_dl_bmi_delta      0.094943
ol_box_delta         0.063521
ol_plus_box_delta    0.047449
box_weight          -0.058631
box_bmi             -0.081560
pass                 1.000000
Name: pass, dtype: float64

### Compare new features to final features in model

We want to see if there's too much cross-correlation between our new features and our extant useful ones:

In [21]:
motion_cols=get_motion_cols(train_data.columns)
momentum_cols=get_momentum_cols(train_data.columns)
train_data=create_momentum_index(train_data, momentum_cols)
train_data=motion_complexity_score(train_data, motion_cols)
final_features=['xpass_situational',  'QB_RB1_offset','off_xpass','n_offense_backfield','motion-momentum','neg_Formations', 'mean_pairwise_dist']

Creat a few more composite features, trying to reconcile box count & DL/box weight:

In [22]:
train_data['box_ewm_dl_weight'] = train_data['box_ewm']*train_data['mean_DL_weight']
train_data['box_ewm_dl_bmi'] = train_data['box_ewm']*train_data['mean_DL_bmi']
train_data['box_ewm_weight'] = train_data['box_ewm']*train_data['box_weight']
train_data['box_ewm_bmi'] = train_data['box_ewm']*train_data['box_bmi']

Initial takeaway is that rolling box EWM, paired with mean DL BMI, tells us  most about pass likelihood:

In [23]:
train_data[final_features+['mean_DL_weight','box_ewm','box_ewm_dl_weight','box_ewm_weight',
                           'box_ewm_dl_bmi','box_ewm_bmi','pass']].corr()

,xpass_situational,QB_RB1_offset,off_xpass,n_offense_backfield,motion-momentum,neg_Formations,mean_pairwise_dist,mean_DL_weight,box_ewm,box_ewm_dl_weight,box_ewm_weight,box_ewm_dl_bmi,box_ewm_bmi,pass
xpass_situational,1.000000,0.092432,0.123280,-0.278399,0.055583,-0.459718,0.387330,-0.211035,-0.110936,-0.202856,-0.150310,-0.195527,-0.160550,0.488006
QB_RB1_offset,0.092432,1.000000,-0.040572,-0.210775,0.027452,-0.133448,0.115273,0.003140,-0.006314,-0.002161,0.003401,0.003623,0.004741,0.097237
off_xpass,0.123280,-0.040572,1.000000,-0.096844,0.053168,-0.157291,0.122287,-0.062612,-0.124522,-0.136016,-0.129566,-0.152558,-0.142943,0.162012
n_offense_backfield,-0.278399,-0.210775,-0.096844,1.000000,0.004917,0.341777,-0.321354,0.088789,0.071419,0.105736,0.085896,0.100240,0.088078,-0.312946
motion-momentum,0.055583,0.027452,0.053168,0.004917,1.000000,-0.062888,-0.003989,0.015853,-0.014821,-0.005697,-0.013290,-0.018801,-0.022675,0.237531
neg_Formations,-0.459718,-0.133448,-0.157291,0.341777,-0.062888,1.000000,-0.386887,0.130078,0.110688,0.160685,0.130601,0.164894,0.141055,-0.409078
mean_pairwise_dist,0.387330,0.115273,0.122287,-0.321354,-0.003989,-0.386887,1.000000,-0.112187,-0.157140,-0.189400,-0.165220,-0.183135,-0.167846,0.309695
mean_DL_weight,-0.211035,0.003140,-0.062612,0.088789,0.015853,0.130078,-0.112187,1.000000,0.027454,0.550157,0.394867,0.529901,0.342918,-0.104301
box_ewm,-0.110936,-0.006314,-0.124522,0.071419,-0.014821,0.110688,-0.157140,0.027454,1.000000,0.848974,0.911123,0.816046,0.917056,-0.088376
box_ewm_dl_weight,-0.202856,-0.002161,-0.136016,0.105736,-0.005697,0.160685,-0.189400,0.550157,0.848974,1.000000,0.971523,0.961886,0.949490,-0.128275


# Pressure Analysis

For each play, we see if any player caused pressure on it:

In [24]:
df_pp_cp = df_player_play.groupby(['gameId','playId']).agg(pressure_play=('causedPressure','any')).reset_index()
df_play = df_play.merge(df_pp_cp,how='left')
df_play.sort_values(by=['gameId','possessionTeam','playId'],inplace=True)

Then, we get the rolling pressure rate for each offensive team in the game:

In [25]:
df_play = pd.concat([df_play,df_play.groupby(['gameId','possessionTeam']).agg(pressure_ct=('pressure_play','cumsum'))],axis=1)
df_play['mean_pr_pre'] = df_play['pressure_ct']/df_play['off_play']
df_play['mean_pressure_ratio'] = df_play.groupby(['gameId','possessionTeam']).mean_pr_pre.shift(1)
df_play['mean_pressure_ratio'] = df_play['mean_pressure_ratio'].fillna(0)

In [28]:
df_play[['gameId','playId','possessionTeam','pressure_play','pressure_ct','off_play','mean_pr_pre','mean_pressure_ratio']].head(11)

,gameId,playId,possessionTeam,pressure_play,pressure_ct,off_play,mean_pr_pre,mean_pressure_ratio
0,2022090800,56,BUF,False,0,1,0.000000,0.000000
1,2022090800,80,BUF,True,1,2,0.500000,0.000000
2,2022090800,101,BUF,False,1,3,0.333333,0.500000
3,2022090800,122,BUF,True,2,4,0.500000,0.333333
4,2022090800,167,BUF,False,2,5,0.400000,0.500000
5,2022090800,191,BUF,False,2,6,0.333333,0.400000
6,2022090800,212,BUF,False,2,7,0.285714,0.333333
7,2022090800,236,BUF,False,2,8,0.250000,0.285714
8,2022090800,529,BUF,False,2,9,0.222222,0.250000
9,2022090800,550,BUF,True,3,10,0.300000,0.222222


### Attempt integration w/OL, box & DL BMI info

By itself, the pressure rate a team faces, somewhat surprisingly, tells us little about pass rates going forward.

In [29]:
df_play[['mean_pressure_ratio','isDropback']].corr()

,mean_pressure_ratio,isDropback
mean_pressure_ratio,1.000000,0.028856
isDropback,0.028856,1.000000


We thus try and integrate it into our previously engineered Box/OL/DL BMI data:

In [30]:
train_data = train_data.merge(df_play[['gameId','playId','mean_pressure_ratio']],how='left')
train_data['box_pressure']  = 5*train_data['box_ewm']+.05*train_data['mean_pressure_ratio']	
train_data['OL_pc'] = train_data['mean_OL_bmi']*train_data['mean_pressure_ratio']
train_data['DL_pc'] = train_data['mean_DL_bmi']*train_data['mean_pressure_ratio']

This still bears little fruit, so we'll try EWM next:

In [31]:
train_data[['box_ewm_dl_bmi','box_ewm','box_pressure','mean_pressure_ratio','OL_pc','DL_pc','pass']].corr()

,box_ewm_dl_bmi,box_ewm,box_pressure,mean_pressure_ratio,OL_pc,DL_pc,pass
box_ewm_dl_bmi,1.000000,0.816046,0.816012,-0.163413,-0.164394,-0.109610,-0.130304
box_ewm,0.816046,1.000000,0.999997,-0.184349,-0.187531,-0.183749,-0.088376
box_pressure,0.816012,0.999997,1.000000,-0.182030,-0.185216,-0.181444,-0.088366
mean_pressure_ratio,-0.163413,-0.184349,-0.182030,1.000000,0.999005,0.994593,0.025978
OL_pc,-0.164394,-0.187531,-0.185216,0.999005,1.000000,0.993991,0.025762
DL_pc,-0.109610,-0.183749,-0.181444,0.994593,0.993991,1.000000,0.016464
pass,-0.130304,-0.088376,-0.088366,0.025978,0.025762,0.016464,1.000000


In [32]:
abs(train_data.corr()['mean_pressure_ratio']).sort_values(ascending=False).head(12)

mean_pressure_ratio    1.000000
OL_pc                  0.999005
DL_pc                  0.994593
box_ewm                0.184349
box_pressure           0.182030
box_ewm_pre            0.176555
box_ewm_weight         0.175361
box_ewm_dl_weight      0.173448
box_ewm_bmi            0.171587
playId                 0.164247
play_id                0.164247
time_remaining         0.163668
Name: mean_pressure_ratio, dtype: float64

# Pressure EWM analysis

Again, like with box count, EWM almost doubles the value of pressure rate to our model

In [34]:
df_play['pressure_ewm_pre'] = df_play.groupby(['gameId','possessionTeam'])['pressure_play'].transform(lambda x: x.ewm(alpha=.1).mean())
df_play['pressure_ewm'] = df_play.groupby(['gameId','possessionTeam']).pressure_ewm_pre.shift(1)

In [35]:
train_data = train_data.merge(df_play[['gameId','playId','pressure_ewm']],how='left')

In [37]:
train_data[['pressure_ewm','pass']].corr()

,pressure_ewm,pass
pressure_ewm,1.000000,0.057406
pass,0.057406,1.000000


In [42]:
train_data = train_data.merge(df_play[['gameId','playId','pressure_ewm']],how='left')
train_data['box_pressure_ewm']  = 5*train_data['box_ewm']+.05+-train_data['pressure_ewm']	
train_data['OL_pc_ewm'] = train_data['mean_OL_bmi']+-train_data['pressure_ewm']
train_data['DL_pc_ewm'] = train_data['mean_DL_bmi']+-train_data['pressure_ewm']

In [43]:
train_data[['box_ewm_dl_bmi','box_ewm','box_pressure_ewm','pressure_ewm','OL_pc_ewm','DL_pc_ewm','pass']].corr()

,box_ewm_dl_bmi,box_ewm,box_pressure_ewm,pressure_ewm,OL_pc_ewm,DL_pc_ewm,pass
box_ewm_dl_bmi,1.000000,0.816046,0.815886,-0.189864,0.189719,0.202260,-0.130304
box_ewm,0.816046,1.000000,0.998529,-0.209743,0.209286,0.209836,-0.088376
box_pressure_ewm,0.815886,0.998529,1.000000,-0.262446,0.261992,0.262514,-0.090406
pressure_ewm,-0.189864,-0.209743,-0.262446,1.000000,-0.999957,-0.999773,0.057406
OL_pc_ewm,0.189719,0.209286,0.261992,-0.999957,1.000000,0.999739,-0.057412
DL_pc_ewm,0.202260,0.209836,0.262514,-0.999773,0.999739,1.000000,-0.059463
pass,-0.130304,-0.088376,-0.090406,0.057406,-0.057412,-0.059463,1.000000
